# Метод ближайших соседей при классификации станций метро по количеству людей которые пьют чай или кофе

## Подготовка данных

In [1]:
import os

# Текущий каталог где находятся файлы
FILES_DIR = "C:\\Projects\\KNN"

# stations.txt содержит данные по станциям на линиях метро, а также соседние станции
# Сначала заполняем матрицу смежности для вершин на линиях метро и соседних станций
metro_stations = {}
with open(os.path.join(FILES_DIR, "stations.txt"), "r", encoding = "utf8") as st:
    m_lines = []
    # Заполняем станции по линиям метро
    for line in st:
        if "***ПЕРЕХОДЫ***" in line:
            break
        if ("***" in line):
            if m_lines:
                # конец линии
                for i in range(len(m_lines)):
                    if m_lines[i] not in metro_stations.keys():
                        metro_stations[m_lines[i]] = {}
                    for j in range(len(m_lines)):
                        metro_stations[m_lines[i]][m_lines[j]] = abs(i-j)
                m_lines=[]
        else:
            m_lines.append(line.split("(")[0].strip().lower())
    
    # Заполняем смежные станции
    for line in st:
        line = line.split("(")[0].strip().lower().replace("+", "/")
        if "/" in line:
            m_lines = line.split("/")
            for i in range(len(m_lines)):
                    if m_lines[i] not in metro_stations.keys():
                        metro_stations[m_lines[i]] = {}
                    for j in range(len(m_lines)):
                        metro_stations[m_lines[i]][m_lines[j]] = 0

with open(os.path.join(FILES_DIR, "metro_by_line.py"), "w", encoding = "utf-8") as f:
    f.write("METRO = {}".format(metro_stations))
    
# Далее на основе данных по линиям метро и смежным станциям строим глобальную матрицу
# применяем алгоритм Дейкстры для поиска оптимального пути между станциями

script_path = os.path.join(FILES_DIR, "Calculate_distances_matrix.py")
%run  $script_path $FILES_DIR
# результат записываем в файл full_dist_matrix.py

## Алгоритмы классификации KNN

In [2]:
import sys
sys.path.append(FILES_DIR)
from full_dist_matrix import METRO

def get_neighbors(k, metro):
    # Код получения соседних станций (включая саму станцию) по k и названию метро из словаря METRO
    neighbours = {}
    for i in METRO[metro]:
    
        if METRO[metro][i] <= k:
            neighbours.update(  {   i : METRO[metro][i]   }   )

    return neighbours

In [3]:
from classes_data import cl_data

def knn(neighbors):
    # В neighbors содержится словарь с данными по соседним станциям и самой станции
    # Изначально используется взвешенное голосование для "чай" и "кофе" в зависимости от расстояния до станции
    votes_tea = 0
    votes_coffee = 0
    
    for i in neighbors:
        if i not in cl_data.keys():
            continue
        else:
            votes_tea += ((1000 - neighbors[i]) / 1000) * cl_data[i]['чай']
            votes_coffee += ((1000 - neighbors[i]) / 1000) * cl_data[i]['кофе']

    if votes_tea > votes_coffee:
        return("чай", votes_tea, votes_coffee)
    elif votes_tea < votes_coffee:
        return("кофе", votes_tea, votes_coffee)

    return ("", votes_tea, votes_coffee)

# Альтернативный алгоритм
def knn2(neighbors):
    # В neighbors содержится словарь с данными по соседним станциям и самой станции
    # Подсчетываем количества людей для классов "чай" и "кофе"
    # Нужно проверить особый случай когда кол-во людей для "чай" и "кофе" одинаковое
    # тогда используем взвешенное голосование (для каждого класса ["чай" и "кофе"] подсчитываем сумму по всем 
    # станциям 1/квадрат расстояния)
    t=0
    c=0
    
    for i in neighbors:
        if i not in cl_data.keys():
            continue
        t += cl_data[i]['чай']
        c += cl_data[i]['кофе']
    if t == c:
        t=0
        c=0
        for i in neighbors:
            if i not in cl_data.keys():
                continue
            t += 1/((1+neighbors[i])**2) * cl_data[i]['чай']
            c += 1/((1+neighbors[i])**2) * cl_data[i]['кофе']

    if t > c:
        return("чай", t, c)
    elif t < c:
        return("кофе", t, c)

    return ("", t, c)

## Ввод данных и выполнение классификации

In [ ]:
k = 0
metro = ''
print("Для выхода нажмите <ENTER>")
while True:
    print("Введите k:")
    k = input()
    if k == '':
        break
    print("Введите название метро:")
    metro = input().strip().lower()
    if metro == '':
        break
    try:
        result = knn2(get_neighbors(int(k), metro))
    except KeyError:
        print("Неправильно указана станция метро")
        continue
    except ValueError:
        print("Число k указано неправильно")
        continue
        
    if not result[0]:
        if result[1] == result[2] == 0:
            print("По станции метро {0} отсутствуют данные".format(metro))
        else:
            print("На станции метро {0} c k={1} чай и кофе пьют поровну (чай = {2}, кофе = {3})"
                  .format(metro, k, result[1], result[2]))
    if result[0] == "чай":
        print("На станции метро {0} с k={1} пьют чай (чай = {2}, кофе = {3})".format(metro, k, result[1], result[2]))
    if result[0] == "кофе":
        print("На станции метро {0} с k={1} пьют кофе (чай = {2}, кофе = {3})".format(metro, k, result[1], result[2]))

Для выхода нажмите <ENTER>
Введите k:
1
Введите название метро:
Комсомольская
На станции метро комсомольская с k=1 пьют кофе (чай = 0, кофе = 1)
Введите k:
1
Введите название метро:
Царицыно
На станции метро царицыно с k=1 пьют чай (чай = 3, кофе = 0)
Введите k:
100
Введите название метро:
проспект мира
На станции метро проспект мира с k=100 пьют кофе (чай = 0.12899548593542098, кофе = 0.4314714033761653)
Введите k:
4
Введите название метро:
Киевская
По станции метро киевская отсутствуют данные
Введите k:
сокол
Введите название метро:
3
Число k указано неправильно
Введите k:
3
Введите название метро:
сокол
На станции метро сокол с k=3 пьют кофе (чай = 2, кофе = 4)
Введите k:
1
Введите название метро:
сокол
На станции метро сокол с k=1 пьют кофе (чай = 2, кофе = 3)
Введите k:
